# Shopify Subdomain Takeover Scanner

**Dataset-based workflow:**
1. Load CSV dataset (`/kaggle/input/all-leads-merged/results.csv`)
2. Filter: Shopify stores + exclude myshopify.com + exclude 200/429
3. Sort by Est Monthly Page Views (highest traffic first)
4. Select range to scan (e.g., rows 1-100)
5. Deep scan with full mode

**Total time:** ~7-9 hours for 10,000 domains (within Kaggle's 12-hour limit)

## Cell 1: Clone Project from GitHub

In [ ]:
%%bash
WORKDIR="/kaggle/working"
PROJECT_DIR="$WORKDIR/subdomain-playground"

echo "=========================================="
echo "Cloning Project from GitHub"
echo "=========================================="

mkdir -p "$WORKDIR"
cd "$WORKDIR"

if [ -d "$PROJECT_DIR" ]; then
    echo "Removing existing copy..."
    rm -rf "$PROJECT_DIR"
fi

git clone --depth 1 https://github.com/sayihhamza/subdomain-playground.git subdomain-playground

if [ ! -d "$PROJECT_DIR" ]; then
    echo "✗ Clone failed!"
    exit 1
fi

cd "$PROJECT_DIR"

echo ""
echo "✓ Project cloned successfully!"
echo ""
echo "Project structure:"
ls -lh | head -20

## Cell 2: Install Python Dependencies

In [ ]:
%%bash
cd /kaggle/working/subdomain-playground

echo "Installing Python requirements..."
python3 -m pip install --quiet -r requirements.txt

echo ""
echo "✓ Dependencies installed!"

## Cell 3: Install Go 1.24

In [ ]:
%%bash
cd /tmp

if command -v sudo >/dev/null 2>&1; then
    SUDO="sudo"
else
    SUDO=""
fi

echo "=========================================="
echo "Installing Go 1.24.1"
echo "=========================================="

$SUDO rm -rf /usr/local/go 2>/dev/null || true

echo "Downloading Go 1.24.1..."
for i in {1..3}; do
    wget -q https://go.dev/dl/go1.24.1.linux-amd64.tar.gz -O /tmp/go.tar.gz && break || sleep 5
done

if [ ! -f /tmp/go.tar.gz ]; then
    echo "✗ Failed to download Go"
    exit 1
fi

$SUDO tar -C /usr/local -xzf /tmp/go.tar.gz
rm -f /tmp/go.tar.gz

echo ""
echo "✓ Go 1.24.1 installed!"
/usr/local/go/bin/go version

## Cell 4: Build Security Tools

Builds 5 tools (5-8 minutes):
- **subfinder**: Passive subdomain enumeration
- **findomain**: Additional passive enumeration (optional)
- **httpx**: HTTP validation
- **dnsx**: DNS + CNAME validation
- **subzy**: Takeover detection

In [ ]:
%%bash
export PATH=/usr/local/go/bin:$PATH
cd /kaggle/working/subdomain-playground

echo "=========================================="
echo "Building Security Tools"
echo "=========================================="
echo ""

mkdir -p bin

# Build Go tools
build_tool() {
    local name=$1
    local repo=$2
    local index=$3
    local total=$4
    local max_attempts=3
    
    echo "[$index/$total] Building $name..."
    
    for attempt in $(seq 1 $max_attempts); do
        if [ $attempt -gt 1 ]; then
            echo "  Retry $attempt/$max_attempts..."
            sleep 2
        fi
        
        if GOBIN=$(pwd)/bin /usr/local/go/bin/go install -v ${repo}@latest 2>&1; then
            if [ -f "bin/$name" ]; then
                echo "  ✓ $name built successfully"
                return 0
            fi
        fi
    done
    
    echo "  ✗ Failed to build $name"
    return 1
}

# Download precompiled binary
download_binary() {
    local name=$1
    local url=$2
    local index=$3
    local total=$4
    local max_attempts=3
    
    echo "[$index/$total] Downloading $name..."
    
    for attempt in $(seq 1 $max_attempts); do
        if [ $attempt -gt 1 ]; then
            echo "  Retry $attempt/$max_attempts..."
            sleep 2
        fi
        
        if curl -sL "$url" -o /tmp/${name}.zip 2>&1; then
            if unzip -q /tmp/${name}.zip -d bin/ 2>&1; then
                if [ -f "bin/$name" ]; then
                    chmod +x "bin/$name"
                    echo "  ✓ $name downloaded successfully"
                    rm -f /tmp/${name}.zip
                    return 0
                fi
            fi
        fi
    done
    
    echo "  ⚠️  $name download failed (optional)"
    rm -f /tmp/${name}.zip
    return 1
}

# Build required tools
build_tool "subfinder" "github.com/projectdiscovery/subfinder/v2/cmd/subfinder" "1" "5"
echo ""

download_binary "findomain" "https://github.com/Findomain/Findomain/releases/latest/download/findomain-linux-i386.zip" "2" "5"
echo ""

build_tool "httpx" "github.com/projectdiscovery/httpx/cmd/httpx" "3" "5"
echo ""

build_tool "dnsx" "github.com/projectdiscovery/dnsx/cmd/dnsx" "4" "5"
echo ""

build_tool "subzy" "github.com/PentestPad/subzy" "5" "5"
echo ""

echo "=========================================="
echo "Verification"
echo "=========================================="

REQUIRED="subfinder httpx dnsx subzy"
TOOLS_OK=true

for tool in $REQUIRED; do
    if [ -f "bin/$tool" ]; then
        echo "  ✓ bin/$tool"
    else
        echo "  ✗ bin/$tool MISSING"
        TOOLS_OK=false
    fi
done

[ -f "bin/findomain" ] && echo "  ✓ bin/findomain (bonus!)" || echo "  ⚠️  bin/findomain (optional)"

if [ "$TOOLS_OK" = false ]; then
    echo ""
    echo "⚠️  Required tools failed - re-run this cell"
    exit 1
fi

echo ""
echo "✓ All required tools built!"

## Cell 5: Set Environment Variables

In [ ]:
import os

os.environ['SUBFINDER_PATH'] = '/kaggle/working/subdomain-playground/bin/subfinder'
os.environ['FINDOMAIN_PATH'] = '/kaggle/working/subdomain-playground/bin/findomain'
os.environ['DNSX_PATH'] = '/kaggle/working/subdomain-playground/bin/dnsx'
os.environ['HTTPX_PATH'] = '/kaggle/working/subdomain-playground/bin/httpx'
os.environ['SUBZY_PATH'] = '/kaggle/working/subdomain-playground/bin/subzy'

os.chdir('/kaggle/working/subdomain-playground')

print("✓ Environment configured")
print(f"✓ Working directory: {os.getcwd()}")

## Cell 6: Load and Sort Dataset

**Configuration:**
- `SORT_BY`: Column to sort by (default: 'Est Monthly Page Views')

**What this cell does:**
1. Loads the full CSV dataset
2. Sorts by specified column (descending)
3. Creates `df_base_filtered` for Cell 7 and Cell 8

**Note:** NO filtering here. Cell 7 and Cell 8 apply their own filters independently.

In [ ]:
import pandas as pd

# ========================================
# CONFIGURATION
# ========================================
SORT_BY = 'Est Monthly Page Views'  # Column to sort by

print("=" * 80)
print("LOADING AND SORTING DATASET")
print("=" * 80)
print("")

# Load CSV from Kaggle input
csv_path = '/kaggle/input/all-leads-merged/results.csv'
print(f"Loading: {csv_path}")
df = pd.read_csv(csv_path, low_memory=False)
print(f"✓ Loaded {len(df):,} total rows")
print("")

# Sort by specified column (NO FILTERING)
print(f"Sorting by: {SORT_BY} (descending)")
if SORT_BY in df.columns:
    # Convert to numeric if it's a numeric column
    df['_sort_numeric'] = pd.to_numeric(
        df[SORT_BY].astype(str).str.replace(r'[^\d.]', '', regex=True),
        errors='coerce'
    ).fillna(0)
    df_sorted = df.sort_values('_sort_numeric', ascending=False)
    print(f"✓ Sorted by {SORT_BY}")
else:
    print(f"⚠️  Column '{SORT_BY}' not found - using unsorted")
    df_sorted = df
print("")

# Store base dataset for Cell 7 (preview) and Cell 8 (scan)
df_base_filtered = df_sorted.copy()

print("=" * 80)
print("✓ Dataset loaded and sorted")
print(f"Total available: {len(df_base_filtered):,} rows")
print("")
print("Next steps:")
print("  • Cell 7: Apply filters + preview range")
print("  • Cell 8: Apply filters + scan range")
print("=" * 80)

## Cell 7: Preview Dataset (Manual Review)

**INDEPENDENT RANGE CONTROL:**
- Configure `PREVIEW_START_ROW` and `PREVIEW_END_ROW` below
- Works from the sorted dataset (`df_base_filtered` from Cell 6)

**Filters applied (for preview only):**
1. Is_Shopify == 'Yes'
2. Exclude *.myshopify.com domains
3. Exclude HTTP 200 and 429

**Use this to:**
- Verify the filtered data looks correct before scanning
- Check specific subdomains manually
- Preview different ranges than what you scan in Cell 8

In [ ]:
import pandas as pd

# ========================================
# PREVIEW CONFIGURATION (Cell 7 only)
# ========================================
PREVIEW_START_ROW = 1       # First row to preview (1-indexed)
PREVIEW_END_ROW = 100       # Last row to preview

print("=" * 120)
print(f"PREVIEW: Rows {PREVIEW_START_ROW} to {PREVIEW_END_ROW} from Sorted Dataset")
print("=" * 120)
print("")
print(f"Source: df_base_filtered from Cell 6 (sorted by {SORT_BY})")
print(f"Total available: {len(df_base_filtered):,} rows")
print("")

# Select preview range from base dataset (sorted, but unfiltered)
df_preview = df_base_filtered.iloc[PREVIEW_START_ROW-1:PREVIEW_END_ROW].copy()
print(f"Selected range: {len(df_preview)} rows")
print("")

# Apply preview filters (for display only)
print("Applying filters for preview...")

# Filter A: Shopify stores only
is_shopify = df_preview['Is_Shopify'] == 'Yes'
print(f"  After Shopify filter: {is_shopify.sum()} rows")

# Filter B: Exclude raw 'myshopify.com' domains
is_custom_domain = ~df_preview['Subdomain'].str.contains('myshopify.com', case=False, na=False)
print(f"  After myshopify.com exclusion: {(is_shopify & is_custom_domain).sum()} rows")

# Filter C: Exclude HTTP 200 and 429
excluded_statuses = [200, 429, '200', '429', 200.0, 429.0]
is_interesting_status = ~df_preview['HTTP_Status'].isin(excluded_statuses)
print(f"  After status exclusion: {(is_shopify & is_custom_domain & is_interesting_status).sum()} rows")

# Apply all filters
df_preview_filtered = df_preview[is_shopify & is_custom_domain & is_interesting_status].copy()
print("")

# Prepare display with original row numbers
display_df = df_preview_filtered.copy()
# Calculate actual row numbers from base dataset
actual_rows = []
for idx in display_df.index:
    actual_row = df_base_filtered.index.get_loc(idx) + 1
    actual_rows.append(actual_row)
display_df.insert(0, 'Row', actual_rows)

# Select columns to display
display_columns = ['Row', 'Subdomain', 'HTTP_Status', SORT_BY, 'CNAME_Record']
display_columns = [col for col in display_columns if col in display_df.columns]

print("=" * 120)
print(f"Displaying {len(display_df)} domains after filters:")
print("=" * 120)
print("")

# Configure pandas display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.width', 120)

# Display
print(display_df[display_columns].to_string(index=False))

print("")
print("=" * 120)
print("📋 Preview Summary:")
print(f"  • Preview range: Rows {PREVIEW_START_ROW} to {PREVIEW_END_ROW}")
print(f"  • After filters: {len(display_df)} domains")
print(f"  • Sorted by: {SORT_BY} (highest first)")
print(f"  • Filters: Shopify + Custom domain + Exclude status 200/429")
print("")
print("💡 NOTE: Cell 8 scans ALL domains in range (scan.py applies its own filters)")
print("=" * 120)

## Cell 8: Deep Scan Execution

**INDEPENDENT RANGE CONTROL:**
- Configure `SCAN_START_ROW` and `SCAN_END_ROW` below
- Works from the sorted dataset (`df_base_filtered` from Cell 6)

**NO FILTERS APPLIED HERE** - scan.py handles all filtering:
- `--require-cname-contains shopify` (checks CNAME chain)
- `--filter-status 3*,4*,5*` (only 3xx/4xx/5xx statuses)

**Scan configuration:**
- Mode: `full` (active + passive enumeration)
- Workers: 2 (optimized for Kaggle)

**Note:** Since inputs are already subdomains, the scanner will skip enumeration and go directly to DNS/HTTP validation

**Estimated time:** 7-9 hours for 10k domains

In [ ]:
import subprocess
import sys
import pandas as pd
import os

# ========================================
# SCAN CONFIGURATION (Cell 8 only)
# ========================================
SCAN_START_ROW = 1       # First row to scan (1-indexed)
SCAN_END_ROW = 100       # Last row to scan

print("=" * 80)
print("PREPARING SCAN")
print("=" * 80)
print("")
print(f"Source: df_base_filtered from Cell 6 (sorted by {SORT_BY})")
print(f"Total available: {len(df_base_filtered):,} rows")
print("")

# Select scan range from base dataset (NO FILTERING - scan.py handles it)
df_scan = df_base_filtered.iloc[SCAN_START_ROW-1:SCAN_END_ROW].copy()
print(f"Scan range: Rows {SCAN_START_ROW} to {SCAN_END_ROW}")
print(f"Selected: {len(df_scan)} domains")
print("")

# Save ALL domains from range to file (no filtering)
targets_file = '/kaggle/working/subdomain-playground/data/all_sources.txt'
df_scan['Subdomain'].to_csv(targets_file, index=False, header=False)
print(f"✓ Saved {len(df_scan)} domains to: {targets_file}")
print("")
print("💡 NOTE: No filters applied here - scan.py will filter during scan")
print("")

# Change to project directory
os.chdir('/kaggle/working/subdomain-playground')

# Add bin to PATH
os.environ['PATH'] = f"/kaggle/working/subdomain-playground/bin:{os.environ['PATH']}"

# Set tool paths
os.environ['SUBFINDER_PATH'] = '/kaggle/working/subdomain-playground/bin/subfinder'
os.environ['FINDOMAIN_PATH'] = '/kaggle/working/subdomain-playground/bin/findomain'
os.environ['DNSX_PATH'] = '/kaggle/working/subdomain-playground/bin/dnsx'
os.environ['HTTPX_PATH'] = '/kaggle/working/subdomain-playground/bin/httpx'
os.environ['SUBZY_PATH'] = '/kaggle/working/subdomain-playground/bin/subzy'

print("=" * 80)
print("STARTING FULL SCAN - SHOPIFY TAKEOVER DETECTION")
print("=" * 80)
print("")

# Count domains
with open('data/all_sources.txt', 'r') as f:
    domain_count = len(f.readlines())

print(f"Total domains: {domain_count}")
print("Workers: 2 (optimized for Kaggle 2-core CPU)")
print("Mode: FULL (active + passive enumeration)")
print("Tools: subfinder + findomain + dnsx + httpx + subzy")
print("")
print("Filters applied by scan.py:")
print("  ✓ CNAME chain contains 'shopify'")
print("  ✓ HTTP status: 3xx, 4xx, 5xx (takeover indicators)")
print("")
print("What this finds:")
print("  ✓ Any subdomain with CNAME pointing to Shopify")
print("  ✓ Checks entire CNAME chain (not just first hop)")
print("  ✓ Catches Shopify behind Cloudflare/CDN")
print("  ✓ Only shows domains with takeover-indicating HTTP status")
print("")
print("Estimated time: 7-9 hours")
print("✓ Completes within Kaggle 12-hour limit with safety margin")
print("")
print("=" * 80)
print("")

# Run scan with real-time output
process = subprocess.Popen(
    [sys.executable, '-u', 'scan.py',
     '-l', 'data/all_sources.txt',
     '--mode', 'full',
     '--require-cname-contains', 'shopify',
     '--filter-status', '3*,403,404,409,41*,43*,5*',
     '--workers', '2'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

# Stream output line by line in real-time
for line in process.stdout:
    print(line, end='', flush=True)

process.wait()
print("")
print("=" * 80)
print(f"Scan completed with return code: {process.returncode}")
print("=" * 80)

## Cell 9: View Results

In [ ]:
import json
from pathlib import Path

results_file = Path("/kaggle/working/subdomain-playground/data/scans/shopify_takeover_candidates.json")

if results_file.exists():
    with results_file.open("r") as f:
        results = json.load(f)

    print("=" * 80)
    print("SCAN RESULTS")
    print("=" * 80)
    print("")
    print(f"Total candidates found: {len(results)}")

    # Count by risk level
    risk_counts = {}
    for r in results:
        risk = r.get("risk_level", "unknown")
        risk_counts[risk] = risk_counts.get(risk, 0) + 1

    print("")
    print("Breakdown by risk:")
    for risk, count in sorted(risk_counts.items()):
        print(f"  {risk.upper()}: {count}")

    print("")
    print("=" * 80)
    print("TOP 10 FINDINGS")
    print("=" * 80)

    sorted_results = sorted(results, key=lambda x: x.get("confidence_score", 0), reverse=True)

    for i, r in enumerate(sorted_results[:10], 1):
        print("")
        print(f"{i}. {r['subdomain']}")
        print(f"   CNAME: {r.get('cname', 'N/A')}")
        print(f"   HTTP Status: {r.get('http_status', 'N/A')}")
        print(f"   Risk: {r.get('risk_level', 'N/A')}")
        print(f"   Confidence: {r.get('confidence_score', 0)}")
else:
    print(f"✗ Results file not found: {results_file}")
    print("Make sure the scan completed successfully.")

## Cell 10: Export to CSV

In [ ]:
import json
import pandas as pd
from pathlib import Path

results_file = Path("/kaggle/working/subdomain-playground/data/scans/shopify_takeover_candidates.json")

if not results_file.exists():
    print("✗ Results file not found. Run the scan first.")
else:
    with results_file.open("r") as f:
        results = json.load(f)

    if not results:
        print("✗ No results to export.")
    else:
        df = pd.DataFrame(results)

        # Select key columns
        columns = [
            "subdomain", "cname", "http_status", "risk_level", "confidence_score",
            "cname_chain_count", "final_cname_target", "a_records", "provider"
        ]
        df_export = df[[col for col in columns if col in df.columns]]
        df_export = df_export.sort_values("confidence_score", ascending=False)

        # Save to CSV
        output_csv = Path("/kaggle/working/shopify_takeovers.csv")
        df_export.to_csv(output_csv, index=False)

        print("=" * 80)
        print("CSV EXPORT")
        print("=" * 80)
        print("")
        print(f"✓ Exported {len(df_export)} results to: {output_csv}")
        print("")
        print("Preview (top 20):")
        print(df_export.head(20).to_string(index=False))
        
        # High-risk only
        df_high = df_export[df_export["risk_level"].isin(["critical", "high"])]
        if len(df_high) > 0:
            high_risk_csv = Path("/kaggle/working/shopify_high_risk.csv")
            df_high.to_csv(high_risk_csv, index=False)
            print("")
            print(f"✓ High-risk only ({len(df_high)} results): {high_risk_csv}")
        
        print("")
        print("=" * 80)
        print("✅ COMPLETE!")
        print("=" * 80)